# Imports

In [174]:

import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import sqlite3 as lite
import sys
import warnings
warnings.filterwarnings('ignore')

In [175]:
plt.style.use('fivethirtyeight')
sns.set()
%matplotlib inline
#import dataset
data = pd.read_csv('CandidateSummaryActionSenate.csv')
#trimming unecessary columns

ndata = data[['can_nam','can_off','can_id', 'can_par_aff','ind_ite_con','ind_uni_con', 'ind_con'  ]]


ndata.columns=['candidate name' ,'running for','candidate ID', 'party', 'individual itemized', 'individual unitemized', "total individual contributions" ]
#prepare for cleaning

ndata[1:10]


,candidate name,running for,candidate ID,party,individual itemized,individual unitemized,total individual contributions
1,"EHLER, CHARLES JOSEPH",S,S6CO00234,REP,NaN,NaN,NaN
2,"MOHARIR, MUKUND MARTAND",S,S6CA00709,IND,NaN,NaN,NaN
3,"FRAZIER, RYAN LAVAR",S,S0CO00229,REP,"$237,080.00","$19,321.00","$256,401.00"
4,"GRASSLEY, CHARLES E SENATOR",S,S0IA00028,REP,"$4,414,642.00","$1,218,408.00","$5,633,053.00"
5,"BLUNT, ROY",S,S0MO00183,REP,"$6,444,218.00","$414,311.00","$6,658,523.00"
6,"PEARSON, CECIL WELDON JR",S,S6KY00096,DEM,NaN,NaN,NaN
7,"GATES, MICHELLE",S,S6GA00259,UNK,NaN,NaN,NaN
8,"BEITIKS, MIKELIS VALAINIS",S,S6CA00766,IND,"$2,995.00","$2,639.00","$5,634.00"
9,"BOOZMAN, JOHN",S,S0AR00150,REP,"$2,040,027.00","$112,514.00","$2,152,542.00"


# Bulk Data processing

In [178]:
# FEC dictates donations above $200 to be itemized, 
# but itemized data can also include values smaller than $200.
# To get the real small donation amount, add the amount of recorded itemized amount under $200 with unitemized amount. 


%matplotlib inline
ddata = pd.read_csv('indvis16ae.csv', )
ddata[0:10]



,Donor Lastname,Donor Name,opensecrets_id,amount
0,BERLE,CHRISTIAN,N00037401,50
1,DELANEY,MARTIN MR,N00030759,25
2,ABRAHAMS,MARSHALL,N00030759,2700
3,POWERS,MICHAEL,N00030759,250
4,HUMMEL,ROBERT MR,N00030759,100
5,HUMMEL,ROBERT MR,N00030759,100
6,DOLIFKA,LANI,N00030759,500
7,HUMMEL,ROBERT MR,N00030759,100
8,WHITE,WARD H,N00030759,500
9,YOUNGDON,YUN,N00030759,250


In [180]:
#cleanup of bad data
mask = ddata.amount.map(lambda x: isinstance(x, str))
frame = ddata[mask]
mask2 = frame.amount.map(lambda x: x.isnumeric())
ddata =frame[mask2]


%matplotlib inline
ddata[390:397]




,Donor Lastname,Donor Name,opensecrets_id,amount
391,SMITH,WENDELL MR,C00027466,51
393,STAMPER,JOHN H MR,C00027466,500
395,WALLISON,FRIEDA,N00030759,250
396,FRAZIER,RYAN LAVAR,N00030759,195
397,WYNNE,ALEC MR,N00030759,50
398,CLARK,THOMAS L,N00030759,300
399,BONSER,KATHLEEN,C00478362,1000


In [182]:
#getting data smaller than 200
amount = ddata.amount.map(lambda x: int(x))
mask4 = amount.map(lambda x: x<=200)
smalldonations = ddata[mask4]

%matplotlib inline
smalldonations[0:10]



,Donor Lastname,Donor Name,opensecrets_id,amount
0,BERLE,CHRISTIAN,N00037401,50
1,DELANEY,MARTIN MR,N00030759,25
4,HUMMEL,ROBERT MR,N00030759,100
5,HUMMEL,ROBERT MR,N00030759,100
7,HUMMEL,ROBERT MR,N00030759,100
16,SHERMAN,JONATHAN DR,C00027466,100
18,SIMMS,RICHARD MR,C00027466,51
19,SKAGGS,JOHN R MR,C00027466,50
20,SLOTA,ROMUALD DR,C00027466,51
23,HOLDER,CLAUDIA D MS,C00027466,68


In [183]:
smalldonations.loc[:,'amounti']= smalldonations.amount.map(lambda x: int(x))
candidatesum= smalldonations[['opensecrets_id', 'amounti']]
candidatesum['Small Donation Total'] = smalldonations.groupby(['opensecrets_id'])['amounti'].transform('sum')
candidatesum =candidatesum[['opensecrets_id', 'Small Donation Total']]
candidatesum=candidatesum.drop_duplicates(subset='opensecrets_id', keep="first")

%matplotlib inline
candidatesum[0:5]

,opensecrets_id,Small Donation Total
0,N00037401,3087
1,N00030759,7806
16,C00027466,2760770
87,C00478362,500
89,C00374066,19374


# Idealogy Score Processing

In [184]:
isdata = pd.read_csv('idealogyscore2016.csv')

stitching= pd.read_csv('legislators-current.csv')


# finding unique FECC ids from various files to merge with main candidate info graph
#bioguide_id with opensecrets_id
b =pd.merge(isdata, stitching, on='bioguide_id', how='inner')
ab = b[['last_name','ideology','rank_from_high','opensecrets_id','bioguide_id']]

#opensecrets
stitching= pd.read_csv('IDstitching.csv')
polarity = pd.merge(ab, stitching, left_on='opensecrets_id', right_on='CID', how='inner')

polarity[0:5]


,last_name,ideology,rank_from_high,opensecrets_id,bioguide_id,CID,CRPName,Party,DistIDRunFor,FECCandID
0,Grijalva,0.000000,439,N00025284,G000551,N00025284,"Grijalva, Raul M",D,AZ03,H2AZ07070
1,Norton,0.005965,438,N00001692,N000147,N00001692,"Norton, Eleanor Holmes",D,DC00,H0DC00058
2,Lee,0.015822,437,N00008046,L000551,N00008046,"Lee, Barbara",D,CA13,H8CA09060
3,Ellison,0.096084,435,N00028257,E000288,N00028257,"Ellison, Keith",D,MN05,H6MN05183
4,Chu,0.097032,434,N00030600,C001080,N00030600,"Chu, Judy",D,CA27,H0CA32101


# combining the graphs

In [185]:
#trimming first name off candidate graph
a = np.array(ndata.as_matrix())
for i in range(len(a)):
    t = a[i][0].split(',')
    a[i][0] = t[0]
    
a = pd.DataFrame(a)
a.columns=['Candidate' ,'running for','candidate ID', 'party', 'individual itemized', 'individual unitemized', "total individual contributions" ]

b =pd.merge(a, candidatesum, on= "Candidate", how='outer')

totalmerge=pd.merge(b,polarity, left_on= 'candidate ID', right_on='FECCandID', how='right')
totalmerge=totalmerge[['Candidate','running for','individual itemized','individual unitemized', 
             'total individual contributions', 'Small Donation Total', 'ideology','opensecrets_id','Party']]

%matplotlib inline
totalmerge[0:50]

KeyError: 'Candidate'